In [ ]:
from typing import Annotated
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import htmltabletomd
import re
from bs4 import BeautifulSoup
from openai import OpenAI
import json
import pandas as pd
import htmlmin
from io import StringIO
import sys
import pandas as pd
from IPython.display import display, Markdown

def extract_text_from_dynamic_site(url, wait_time=10):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        print(f"Loaded: {url}")
        time.sleep(wait_time)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        return soup
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        driver.quit()

def get_f1_driver_standings(
    standings_type: Annotated[str, "Select 'drivers' or 'team' championship standings."]
) -> str:
    """Get the current F1 drivers or teams standings. 
    Use it to determine the points for each driver in the driver's championship or the points for each team in the teams' championship."""
    if standings_type=="team":
        url = f"https://www.formula1.com/en/results/2025/team"
    else:
        url = f"https://www.formula1.com/en/results/2025/drivers"
    soup_og = extract_text_from_dynamic_site(url, wait_time=3)
    html_table = soup_og.find_all('table')[0]
    html_text = str(html_table.prettify())
    html_text = re.sub(" class=\"[^\"]*\"", "", html_text)
    html_text = re.sub("<img[^>]*>", "", html_text)
    html_text = re.sub("<a[^>]*>", "", html_text)
    html_text = re.sub("<span[^>]*>", "", html_text)
    html_text = re.sub("</span[^>]*>", "", html_text)
    html_text = re.sub("<br>", "", html_text)
    html_text = re.sub("<p>", "", html_text)
    html_text = re.sub("\\\n *", "", html_text)
    drivers_standings_table = htmltabletomd.convert_table(html_text)
    return drivers_standings_table

# Retreive F1 schedule (with time zones)
def get_f1_schedule() -> str:
    # Parse the F1 schedule from the F1 website
    url = f"https://www.formula1.com/en/racing/2025.html"
    soup_og = extract_text_from_dynamic_site(url, wait_time=3)

In [5]:
# Drivers
url = f"https://www.formula1.com/en/results/2025/drivers"
soup_og = extract_text_from_dynamic_site(url, wait_time=3)
html_table = soup_og.find_all('table')[0]
html_text = str(html_table.prettify())
html_text = re.sub(" class=\"[^\"]*\"", "", html_text)
html_text = re.sub("<img[^>]*>", "", html_text)
html_text = re.sub("<a[^>]*>", "", html_text)
html_text = re.sub("<span[^>]*>", "", html_text)
html_text = re.sub("</span[^>]*>", "", html_text)
html_text = re.sub("<br>", "", html_text)
html_text = re.sub("<p>", "", html_text)
html_text = re.sub("\\\n *", "", html_text)
dr_raw_markdown_table = htmltabletomd.convert_table(html_text)
drivers = pd.read_html(StringIO(html_text))[0]

Loaded: https://www.formula1.com/en/results/2025/drivers


In [6]:
drivers['team_points'] = drivers.groupby('TEAM')['PTS.'].transform('sum')
drivers['%_team_points'] = drivers['PTS.'] / drivers['team_points'] * 100
drivers[drivers['team_points']>50][['DRIVER', '%_team_points']].sort_values(by='%_team_points', ascending=False).head(10)

,DRIVER,%_team_points
2,MaxVerstappenVER,95.041322
6,AlexanderAlbonALB,81.395349
3,GeorgeRussellRUS,74.615385
9,NicoHulkenbergHUL,67.272727
8,IsackHadjarHAD,65.517241
4,CharlesLeclercLEC,58.214286
0,OscarPiastriPIA,52.512156
10,LanceStrollSTR,51.612903
11,FernandoAlonsoALO,48.387097
1,LandoNorrisNOR,47.487844


In [7]:
# HTML table whitespace
dr_html_table = drivers.set_index('POS.')[['DRIVER', 'TEAM', 'PTS.']].to_html(border='')
# Minify HTML
dr_minified = htmlmin.minify(dr_html_table, remove_empty_space=True)
# Markdown table
dr_markdown_table = drivers.set_index('POS.')[['DRIVER', 'TEAM', 'PTS.']].to_markdown()
# Natural language - New line
dr_nl_1 = []
for row in drivers[['POS.', 'DRIVER', 'TEAM', 'PTS.']].values:
    dr_nl_1.append(f"{row[0]}. {row[1]} has {row[3]} points and drives for {row[2]}.")
dr_nl_1 = '\n'.join(dr_nl_1)
# Natural language - <> tag inline
dr_nl_2 = []
for row in drivers[['POS.', 'DRIVER', 'TEAM', 'PTS.']].values:
    dr_nl_2.append(f"<{row[0]}>{row[1]} has {row[3]} points and drives for {row[2]}</{row[0]}>")
dr_nl_2 = ''.join(dr_nl_2)
# Natural language - <> tag new line
dr_nl_3 = []
for row in drivers[['POS.', 'DRIVER', 'TEAM', 'PTS.']].values:
    dr_nl_3.append(f"<{row[0]}>{row[1]} has {row[3]} points and drives for {row[2]}</{row[0]}>")
dr_nl_3 = '\n'.join(dr_nl_3)

In [8]:
dr_table_formats = {
    "html_table": dr_html_table,
    "html_mini": dr_minified,
    "markdown_table": dr_markdown_table,
    "raw_markdown_table": dr_raw_markdown_table,
    "natural_language_new_line": dr_nl_1,
    "natural_language_inline_tags": dr_nl_2,
    "natural_language_new_line_tags": dr_nl_3
}

In [ ]:
# Teams
url = f"https://www.formula1.com/en/results/2025/team"
soup_og = extract_text_from_dynamic_site(url, wait_time=3)
html_table = soup_og.find_all('table')[0]
html_text = str(html_table.prettify())
html_text = re.sub(" class=\"[^\"]*\"", "", html_text)
html_text = re.sub("<img[^>]*>", "", html_text)
html_text = re.sub("<a[^>]*>", "", html_text)
html_text = re.sub("<span[^>]*>", "", html_text)
html_text = re.sub("</span[^>]*>", "", html_text)
html_text = re.sub("<br>", "", html_text)
html_text = re.sub("<p>", "", html_text)
html_text = re.sub("\\\n *", "", html_text)
tm_raw_markdown_table = htmltabletomd.convert_table(html_text)
teams = pd.read_html(StringIO(html_text))[0]
# HTML table whitespace
tm_html_table = teams.set_index('POS.')[['TEAM', 'PTS.']].to_html(border='')
# Minify HTML
tm_minified = htmlmin.minify(tm_html_table, remove_empty_space=True)

Loaded: https://www.formula1.com/en/results/2025/team


In [10]:
# HTML table whitespace
tm_html_table = teams.set_index('POS.')[['TEAM', 'PTS.']].to_html(border='')
# Minify HTML
tm_minified = htmlmin.minify(tm_html_table, remove_empty_space=True)
# Markdown table
tm_markdown_table = teams.set_index('POS.')[['TEAM', 'PTS.']].to_markdown()
# Natural language - New line
tm_nl_1 = []
for row in teams[['POS.', 'TEAM', 'PTS.']].values:
    tm_nl_1.append(f"{row[0]}. {row[1]} has {row[2]} points.")
tm_nl_1 = '\n'.join(tm_nl_1)
# Natural language - <> tag inline
tm_nl_2 = []
for row in teams[['POS.', 'TEAM', 'PTS.']].values:
    tm_nl_2.append(f"<{row[0]}>{row[1]} has {row[2]} points</{row[0]}>")
tm_nl_2 = ''.join(tm_nl_2)
# Natural language - <> tag new line
tm_nl_3 = []
for row in teams[['POS.', 'TEAM', 'PTS.']].values:
    tm_nl_3.append(f"<{row[0]}>{row[1]} has {row[2]} points</{row[0]}>")
tm_nl_3 = '\n'.join(tm_nl_3)

In [11]:
tm_table_formats = {
    "html_table": tm_html_table,
    "html_mini": tm_minified,
    "markdown_table": tm_markdown_table,
    "raw_markdown_table": tm_raw_markdown_table,
    "natural_language_new_line": tm_nl_1,
    "natural_language_inline_tags": tm_nl_2,
    "natural_language_new_line_tags": tm_nl_3
}

In [12]:
table_formats = list(set(dr_table_formats.keys()).intersection(set(tm_table_formats.keys())))

In [13]:
qas = pd.read_csv('test_prompts.txt', sep='|', header=None, names=['Prompt', 'Answer']).values

In [101]:
results = []
for i, (prompt, answer) in enumerate(qas):
    for curr_table_name in table_formats:
        # Point to the local server
        client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
        model = "qwen/qwen3-4b-2507"
        def get_f1_driver_standings() -> str:
            """Get the current F1 drivers standings. 
            Use it to determine the points for each driver in the driver's championship."""
            return dr_table_formats[curr_table_name]
        def get_f1_team_standings() -> str:
            """Get the current F1 teams standings. 
            Use it to determine the points for each team in the teams' championship."""
            return tm_table_formats[curr_table_name]

        tools = [
            {
                "type":"function",
                "function": {
                    "name": "get_f1_driver_standings",
                    "description": "Get the current F1 drivers standings. Use it to determine the points for each driver in the driver's championship.",
                    "parameters":  {
                        "type": "object",
                        "properties": {}
                    },
                        "additionalProperties": False,
                    },
                },
            {
                "type":"function",
                "function": {
                    "name": "get_f1_team_standings",
                    "description": "Get the current F1 team standings. Use it to determine the points for each team in the teams' championship.",
                    "parameters":  {
                        "type": "object",
                        "properties": {}
                    },
                        "additionalProperties": False,
                    },
                },
        ]

        messages = [
            {
                "role":"system",
                "content":"""You are an expert in Formula 1 that will assist the user with questions regarding the sport.
                The current season is 2025, there have been a few changes in terms of driver line-ups.
                If you need to know which driver belongs to a certain team or which team a driver belongs to, use `get_f1_driver_standings`.
                Use `get_f1_driver_standings` and `get_f1_team_standings` to assist the user.""",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ]

        # LM Studio
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
        )
        first_response = response
        # Extract the arguments for get_delivery_date
        # Note this code assumes we have already determined that the model generated a function call.
        tool_calls = getattr(response.choices[0].message, "tool_calls", None)
        if tool_calls and len(tool_calls) > 0:
            tool_call_response_payloads = []
            function_call_results = {}
            for tool_call in tool_calls:
                # Get a reference to the current module
                current_module = sys.modules[__name__]
                # Dynamically get and call the function
                function_name = tool_call.function.name
                result = getattr(current_module, function_name)()
                tool_call_response_payload = {
                    "id":tool_call.id,
                    "type":tool_call.type,
                    "function":tool_call.function,
                }
                tool_call_response_payloads.append(tool_call_response_payload)
                function_call_results[function_name] = result
            assistant_tool_call_request_message = {
                "role": "assistant",
                "tool_calls": tool_call_response_payloads
            }

            # Create a message containing the result of the function call
            function_call_result_message = {
                "role":"tool",
                "content": json.dumps(
                    function_call_results
                ),
                "tool_call_id": tool_call.id,
            }

            # Prepare the chat completion call payload
            completion_messages_payload = [
                messages[0],
                messages[1],
                assistant_tool_call_request_message,
                function_call_result_message,
            ]

            # Call the OpenAI API's chat completions endpoint to send the tool call result back
            # to the model
            response = client.chat.completions.create(
                model=model,
                messages=completion_messages_payload,
            )

        final_response = response.choices[0].message.content
        messages = [
            {
                "role":"system",
                "content":"You a teacher evaluating student answers to questions about Formula 1. You have a <question> a <correct_answer> and a <student_answer>. Only respond with 'Correct' or 'Incorrect'.",
            },
            {
                "role": "user",
                "content": f"<question>{prompt}</question><correct_answer>{answer}</correct_answer><student_answer>{final_response}</student_answer>",
            },
        ]

        # LM Studio
        response = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        evaluation = response.choices[0].message.content
        results.append([model, curr_table_name, prompt, answer, final_response, len(final_response), evaluation])
        print(f"{i+1}/{len(qas)} - {curr_table_name}", end='\r')
results_df = pd.DataFrame(results, columns=['model', 'table_format', 'prompt', 'correct_answer', 'final_answer', 'final_answer_length', 'evaluation'])
results_df['good'] = results_df['evaluation'].map({'Correct': 1, 'Incorrect': 0})

In [102]:
results_df.groupby('table_format')[['good', 'final_answer_length']].mean().reset_index()

,table_format,good,final_answer_length
0,html_mini,0.666667,617.888889
1,html_table,0.555556,532.222222
2,markdown_table,0.555556,449.777778
3,natural_language_inline_tags,0.444444,365.444444
4,natural_language_new_line,0.555556,547.222222
5,natural_language_new_line_tags,0.555556,483.555556
6,raw_markdown_table,0.666667,553.333333


In [127]:
results_df[results_df['table_format']=='html_mini'][['prompt', 'good']]

,prompt,good
0,Who is leading the f1 drivers championship?,1
7,Which driver has the largest share of his team...,0
14,Which driver has the largest share of his team...,0
21,"Which driver who is NOT part of Mclaren, Redbu...",1
28,Which team is in 3rd place in the championship?,1
35,Which drivers are part of the team that is las...,1
42,Which drivers have over 100 points in the stan...,0
49,Who is the worst driver racing for Alpine?,1
56,What is the points difference between 1st and ...,1


In [103]:
results_df.groupby('prompt')[['good', 'final_answer_length']].mean()

,good,final_answer_length
prompt,,
What is the points difference between 1st and 2nd place?,1.000000,147.857143
"Which driver has the largest share of his teams points, where the combined team total is more than 50 points?",0.000000,396.285714
Which driver has the largest share of his teams points?,0.428571,1965.000000
"Which driver who is NOT part of Mclaren, Redbull, Ferrari or Mercedes has the most points?",0.714286,651.285714
Which drivers are part of the team that is last in the standings?,0.285714,238.285714
Which drivers have over 100 points in the standings?,0.142857,432.857143
Which team is in 3rd place in the championship?,1.000000,62.571429
Who is leading the f1 drivers championship?,1.000000,79.857143
Who is the worst driver racing for Alpine?,0.571429,589.571429


In [ ]:
results_df[(results_df['table_format']==table_formats[tf])].groupby('prompt')[['good', 'final_answer_length']].mean()

In [131]:
tf = 0
pmpt = 6
print(table_formats[tf])
print(qas[pmpt])
display(Markdown(results_df[(results_df['table_format']==table_formats[tf]) & (results_df['prompt']==qas[pmpt][0])]['final_answer'].iloc[0]))

html_mini
['Which drivers have over 100 points in the standings?'
 'Oscar Piastri, Lando Norris, Max Verstappen, George Russell, Charles Leclerc and Lewis Hamilton']


The drivers with over 100 points in the standings are:

1. **Oscar Piastri** (McLaren) - 324 points  
2. **Lando Norris** (McLaren) - 293 points  
3. **Max Verstappen** (Red Bull Racing) - 230 points  
4. **George Russell** (Mercedes) - 194 points  
5. **Charles Leclerc** (Ferrari) - 163 points  
6. **Lewis Hamilton** (Ferrari) - 117 points  

Thus, the drivers with over 100 points are: **Oscar Piastri, Lando Norris, Max Verstappen, George Russell, Charles Leclerc, and Lewis Hamilton**.